# Jupyter Notebooks in Mathematics for Machine Learning - Linear Algebra MOOC


## “I hadn't used Python earlier, but the programming assignments (especially the PageRank algorithm implementation) have motivated me to start learning the language.” - Learner Feedback

In [9]:
# Options are: 'LA', 'MVC' or 'PCA'
course = 'LA'

In [10]:
from datetime import datetime as dt
import numpy as np
import pandas as pd
import matplotlib as plt
import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls

import IPython.core.display as di

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Show / Hide Code</button>''', raw=True)


py.init_notebook_mode(connected=True)

time = dt.now
print(dt.now())

Show / Hide Code

2018-11-23 12:39:09.721328


In [11]:

df2 = pd.read_csv('C:\\Users\\ikalkani\\Box\\Digital Learning Hub\\10. Data Analytics\Data\Coursera-M4ML\\Linear Algebra - Coursera data\\programming_submissions.csv', low_memory=False)
df3 = pd.read_csv('C:\\Users\\ikalkani\\Box\\Digital Learning Hub\\10. Data Analytics\Data\Coursera-M4ML\\Linear Algebra - Coursera data\\programming_assignment_submission_schema_parts.csv', low_memory=False)



df3 = df3.set_index('programming_assignment_id')

df2 = df2.join(df3,'programming_assignment_id',how='left')

gradebook = pd.read_csv('C:\\Users\\ikalkani\\Box\\Digital Learning Hub\\10. Data Analytics\Data\Coursera-M4ML\\Linear Algebra - Coursera data\\gradebook.csv', low_memory=False)
users = pd.read_csv('C:\\Users\\ikalkani\\Box\\Digital Learning Hub\\10. Data Analytics\Data\Coursera-M4ML\\Linear Algebra - Coursera data\\users.csv')
users = users.set_index('imperial_user_id')
country_codes = pd.read_csv('C:\\Users\\ikalkani\\Box\\Digital Learning Hub\\10. Data Analytics\Data\Coursera-M4ML\\Linear Algebra - Coursera data\\ISO_country_codes.csv')
country_codes = country_codes.set_index('alpha2')
## this sets the index of  the country_codes table to the two letter code, making it easy to join with the 'country_cd' column in df

gb = gradebook.join(users, 'Anonymized Coursera ID (imperial_user_id)', how = 'left')


### Pull in additional country information from ISO data so that we have full country name and alpha3 code which is required for the worldview Plotly map
gb =  gb.join(country_codes,'country_cd', how = 'left')
total = gb['Anonymized Coursera ID (imperial_user_id)'].nunique()



In [12]:
def countries (df, column):
    
    df1 = df.groupby('alpha3').count()

    data = [dict(
            type = 'choropleth',
            locations = df1.index,
            z = df1[column],
            colorscale = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']],
            autocolorscale = False,
            marker = dict(
                line = dict (
                    color = 'rgb(180,180,180)',
                    width = 0.5
                ) ) ) ]

    layout = dict(
        height = 800,
        width = 800,
        geo = dict(
            showframe = True,
            showcoastlines = False,
            projection = dict(
                type = 'orthographic'
            )))

    fig = dict( data=data, layout=layout )
    plot = py.iplot(fig)


In [5]:
print('Total number of students to date:', total )
countries (gb, 'Anonymized Coursera ID (imperial_user_id)')


Total number of students to date: 43125


# Total numbers of learners per notebook 

In [6]:
totallearners = df2['imperial_user_id'].nunique()

print('Total number of learners: ', totallearners)

d2 = df2.groupby('programming_assignment_submission_schema_part_title').nunique()

d2['submission_rate'] = d2['programming_submission_id']/d2['imperial_user_id']

d2.drop(d2[d2.imperial_user_id < 100].index, inplace=True)

ordered_index = ['Identifying special matrices', 'Gram Schmidt', 'Reflecting Bear',
       'submission.json']

d2 = d2.reindex(ordered_index)


data = [
    go.Bar(
        x=d2.index, # assign x as the dataframe column 'x'
        y=d2['imperial_user_id'],
        opacity=0.6
    )
]


py.iplot(data, filename='basic-bar')

Total number of learners:  5681


# Attempts per learner

In [7]:

d2 = df2.groupby('programming_assignment_submission_schema_part_title').nunique()

d2['submission_rate'] = d2['programming_submission_id']/d2['imperial_user_id']

d2.drop(d2[d2.imperial_user_id < 100].index, inplace=True)

ordered_index = ['Identifying special matrices', 'Gram Schmidt', 'Reflecting Bear',
       'submission.json']

d2 = d2.reindex(ordered_index)

data = [
    go.Bar(
        x=d2.index, # assign x as the dataframe column 'x'
        y=d2['submission_rate'],
        opacity=0.6
    )
]


py.iplot(data, filename='basic-bar')

# Passes / fails

In [8]:
totalattempts = df2['programming_submission_id'].nunique()

ism = df2.loc[df2['programming_assignment_submission_schema_part_title'] == 'Identifying special matrices']
gs = df2.loc[df2['programming_assignment_submission_schema_part_title'] == 'Gram Schmidt']
rb = df2.loc[df2['programming_assignment_submission_schema_part_title'] == 'Reflecting Bear']
fs = df2.loc[df2['programming_assignment_submission_schema_part_title'] == 'submission.json']


def passed(df2):
    pass_fail = df2.loc[:,'programming_submission_score'].groupby(df2.loc[:,'programming_submission_score']).count()
    passed = sum(pass_fail[pass_fail.index >= 8])
    return passed

def failed(df2):
    pass_fail = df2.loc[:,'programming_submission_score'].groupby(df2.loc[:,'programming_submission_score']).count()
    failed = sum(pass_fail[pass_fail.index < 8])
    return failed

# data = [
#     go.Bar(
#             x=['total attempts', 'failed', 'passed'],
#             y=[totalattempts, failed, passed]
#     )
# ]


# py.iplot(data, filename='basic-bar')


trace2 = go.Bar(
    x=['Identifying special matrices', 'Gram Schmidt', 'Reflecting Bear',
       'submission.json'],
    y=[passed(ism),passed(gs),passed(rb),passed(fs)],
    name='Passed',
    marker = dict(
          color = 'blue'),
    opacity=0.6
)
trace1 = go.Bar(
    x=['Identifying special matrices', 'Gram Schmidt', 'Reflecting Bear',
       'submission.json'],
    y=[failed(ism),failed(gs),failed(rb),failed(fs)],
    name='Failed',
    marker = dict(
          color = 'red'),
     opacity=0.6
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='stack'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='stacked-bar')